# Daskified unyt arrays  

Throughout yt, data is stored using unit-aware [unyt](https://unyt.readthedocs.io/en/stable/) arrays. A unyt array is a subclass of a standard numpy ndarray wrapped with operations that track units. So a part of the Daskening of yt relies on adding Dask support to unyt arrays ([PR 185](https://github.com/yt-project/unyt/pull/185)). As this has potential users beyond yt users, it is worth walking through its usage. We also describe the general approach to implementing Dask support for unyt arrays.

## example usage 

At present, the primary way to create a `unyt_dask_array` is through the `unyt_from_dask` function, which accepts a standard Dask array and a unit along with all of the optional parameters for `unyt.unyt_array`:

In [1]:
from unyt import dask_array as unyt_dask_array, unyt_quantity, unyt_array
from dask import array as da
import numpy as np

In [2]:
x1 = unyt_dask_array.unyt_from_dask(da.random.random((1e6,), chunks=(1e5)), 'm')
x1

unyt_dask_array<random_sample, shape=(1000000,), dtype=float64, chunksize=(100000,), chunktype=numpy.ndarray, units=m>

So we've created what looks like a Dask array of 10 chunks, but with an extra units attribute. Like a `unyt_array`, we can convert units:

In [3]:
x1.to('cm')

unyt_dask_array<mul, shape=(1000000,), dtype=float64, chunksize=(100000,), chunktype=numpy.ndarray, units=cm>

and we can operate on multiple arrays, and any necessary unyt conversions will be applied automatically:

In [4]:
x2 = unyt_dask_array.unyt_from_dask(0.001 * da.random.random((1e6,), chunks=(1e5)), 'km')

In [5]:
x = (x1 + x2).to('m')
x

unyt_dask_array<mul, shape=(1000000,), dtype=float64, chunksize=(100000,), chunktype=numpy.ndarray, units=m>

but in both these cases, we're still working with delayed arrays. Once we call `compute`, we'll get back either a plain `unyt_array` or `unyt_quanity` depending on the operation:

In [6]:
x.mean().compute()

unyt_quantity(0.99956126, 'm')

In [7]:
x[50:60].compute()

unyt_array([0.895046  , 0.9392074 , 0.98590878, 0.5823761 , 0.26222106,
            1.74925792, 0.41219676, 0.38931168, 0.71809784, 0.44802312], 'm')

## Design

Designing the `unyt_dask_array` is an interesting problem. A standard Dask array is what's called a [Dask Collection--broken link](insertlink) and it implements a large number of array operations to allow computation over a chunked array. A `unyt_array` is a direct subclass of a `numpy.ndarray` with implementations of `numpy` array protocols in order to wrap operations in units and specify the correct units behavior for different operations. Given that the cross-chunk operations implemented by Dask are fairly complex, we created the `unyt_dask_array` as a subclass of a standard Dask array. All of the units-related operations are then handled using hidden `unyt_quantity` attributes and decorators and anytime the units-related operation indicates a unit conversion, those conversion factors are multiplied onto the dask array. 

## A performance comparison

Using unyt dask arrays comes with the enhanced performance expected from using dask arrays, but it's important that ensure that our extra tracking of units is not significantly undermining performance relative to normal dask arrays. 

We can do some initial testing by creating arrays for each of our array flavors, a plain `numpy.ndarray`, a plain `unyt.unyt_array`, a plain `dask.array` and a `unyt_dask_array`, and comparing execution time of an operation that causes a change in units:

In [8]:
array_shape = (int(1e8), )
chunk_size = 1e6

plain_numpy = np.ones(array_shape[0])
plain_unyt = unyt_array(plain_numpy,'m')
plain_dask = da.ones(array_shape[0], chunks = (chunk_size,))
unyt_dask = unyt_dask_array.unyt_from_dask(plain_dask,'m')

And for each of our arrays, we'll compute the time:

In [9]:
%%timeit 
(plain_numpy ** 2).mean()

215 ms ± 9.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit 
(plain_unyt ** 2).mean()

209 ms ± 6.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit 
(plain_dask ** 2).mean().compute()

79.7 ms ± 3.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit 
(unyt_dask ** 2).mean().compute()

70.5 ms ± 2.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Operations with unit conversions will incur an extra penalty:

In [13]:
%%timeit 
(plain_unyt.to('cm') ** 2).mean()

368 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit 
(unyt_dask.to('cm') ** 2).mean().compute()

101 ms ± 414 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


but we can see that (1) better performance for `unyt_dask` arrays than plain `unyt_array` and (2) similar performance for our `unyt_dask` and plain `dask.array` arrays. We show some more complete performance testing below.

### An aside on when to convert units

As a small aside, it's worth a reminder that when stringing together operations you can sometimes save on computation by delaying the scalar operation until after any reductions. Every unit conversion is an elementwise-multiplication of a constant and so if that multiplciation by a constant can be delayed until after a reduction, you can save on the number of multiplications. For example, the following operations are equivalent:

In [15]:
result = ( ( 100 * plain_numpy )** 2).mean()
result_convert_after = (plain_numpy** 2).mean() * (100 **2)

print([result, result_convert_after, result == result_convert_after])

[10000.0, 10000.0, True]


since our unit conversions are simply scalar multiplications, the unit equivalent would be:

In [16]:
result = (plain_unyt.to('cm')** 2).mean()
result_convert_after = (plain_unyt** 2).mean().to('cm * cm') 

print([result, result_convert_after, result == result_convert_after])

[unyt_quantity(10000., 'cm**2'), unyt_quantity(10000., 'cm**2'), array(True)]


So when converting units, converting after a computation:

In [17]:
%%timeit 
(plain_unyt ** 2).mean().to('cm*cm')

211 ms ± 7.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


will be faster than converting before the operation:

In [18]:
%%timeit 
(plain_unyt.to('cm') ** 2).mean()

376 ms ± 8.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


and in the case of our `unyt_dask` arrays:

In [19]:
%%timeit 
(unyt_dask ** 2).to('cm*cm').mean().compute()

104 ms ± 3.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%timeit 
(unyt_dask.to('cm') ** 2).mean().compute()

108 ms ± 4.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


So we see that in operations where it's possible, it is worth putting off unit conversions until after array reductions so that we spend less time on multiplying by constants. 

### Final performance comparison

Returning to the general question of unyt-dask array performance, we want to be sure that our unyt-dask arrays are preforming on-par with standard dask arrays. Towards that end, we've run a suite of performance tests measuring the time to execute `(x ** 2).mean()` vs size of the array, `x`. The full code is available at [`/code/test_daskunyt.py`](https://github.com/chrishavlin/scipy2021_ytDaskening/blob/main/code/test_daskunyt.py) and the following figure captures the results:

![](images/dask_unyt_perf.png)

The y-axis is the minimum execution time of the operation, x-axis is the size of the array. The black and red curves are standard numpy and unyt arrays, respectively. The blue and green curves are dask and unyt-dask arrays for different number of workers (4 and 6, both single-threaded). The chunksize is fixed at 1e7 for all runs. So we can see that at small array sizes, standard `numpy` and `unyt_array` arrays are faster (not surprising), but as array size increases, the `dask` and `unyt_dask` arrays are faster. We can also see that the unyt-dask arrays perform similarly to plain dask arrays and at larger array sizes provide a decent speedup compared to plain unyt arrays. Furthermore, the unyt-dask arrays allows computation on larger-than memory arrays. The largest array tests would require 80 Gb of memory if using a plain unyt array:

In [21]:
unyt_dask_array.unyt_from_dask(da.ones((1e10,), chunks=(1e7)),'m')

unyt_dask_array<ones, shape=(10000000000,), dtype=float64, chunksize=(10000000,), chunktype=numpy.ndarray, units=m>

While the unyt-dask arrays may be quite useful to the general SciPy community, we are experimenting with using them directly within yt where the "chunks" of the dask-arrays are decided using yt's spatial indexing of datasets. We discuss this in the following section.